# Solução - Projeto Automação Web - Busca de Preços

In [43]:
# versao para github

import pandas as pd
from selenium import webdriver

produtos = pd.read_excel("Produtos.xlsx")
produtos = produtos.fillna("-") # substitui os campos NaN por -
display(produtos)


,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,-,-
1,"Smart TV LED PRO 43"" Full HD LG 43LM631C0SB",https://www.amazon.com.br/Smart-LG-3LM631C0SB-...,https://www.americanas.com.br/produto/13444474...,https://www.magazineluiza.com.br/smart-tv-led-...,1979,-,-


In [38]:
def transformar_texto(texto):
    return float(texto.replace("R$", "").replace(".", "").replace(",", "."))

In [37]:
# driver = webdriver.Chrome()

# # pegar preço amazon
# driver.get("https://www.amazon.com.br/Smart-LG-3LM631C0SB-Conversor-Digital/dp/B07WJX6PHL/ref=dp_prsubs_1?pd_rd_i=B07WJX6PHL&psc=1")
# preco_amazon = driver.find_element_by_class_name("a-color-price").text
# transformar_texto(preco_amazon)
# print(preco_amazon)

# #pegar preço lame 
# driver.get("https://www.americanas.com.br/produto/134444740?pfm_carac=smart-tv-led-pro-43-full-hd-lg-43lm631c0sb-thinq-ai-3-hdmi-2-usb-wi-fi-conversor-digital&pfm_index=1&pfm_page=search&pfm_pos=grid&pfm_type=search_page")
# preco_lame = driver.find_element_by_class_name("priceSales").text
# transformar_texto(preco_lame)
# print(preco_lame)

# #pegar preço mglu
# driver.get("https://www.magazineluiza.com.br/smart-tv-led-43-lg-pro-ai-43lm631c0sb-webos-4-5-fullhd-3-hdmi-2usb-preto/p/jf8ec5b9f5/et/elit/")
# preco_mglu = driver.find_element_by_class_name("price-template__text").text
# transformar_texto(preco_mglu)
# print(preco_mglu)



2.199,90
2199.9


In [49]:
# # se quiser que execute em background:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# driver = webdriver.Chrome(options = options)
# # porém nem todos os sites funcionam bem nesse modo escondido

driver = webdriver.Chrome()
# driver.set_window_position(-10000,0) # essa linha esconde o navegador se quiser

enviar_email = False
desconto_min = 0.2

# Duas opções pra percorrer a tabela:

# for i in range(len(produtos['Link Produto'])):
#     print(i)


for i, linha in produtos.iterrows():
    # pegar preço amazon
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_class_name("a-color-price").text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        try:
            # aula 9 projeto 2: Tratando o Caso Especial de 2 Locais de Preço Possíveis
            # https://saucelabs.com/resources/articles/selenium-tips-css-selectors (aula 9 projeto 2)
            preco_amazon = driver.find_element_by_class_name("div.a-section span.a-size-base").text 
            preco_amazon = transformar_texto(preco_amazon)
        except:
            print('Produto () não disponível na Amazon.'.format(linha['Link Produto']))
            preco_amazon = linha['Preço Original'] * 3       # colocando um preço bem caro

    # pegar preço lame 
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name("priceSales").text
        preco_lame = transformar_texto(preco_lame)
    except:
        print('Produto () não disponível nas Lojas Americanas.'.format(linha['Link Produto']))
        preco_lame = linha['Preço Original'] * 3       
        
    # pegar preço mglu
    driver.get(linha['Magazine Luiza'])
    try:
        preco_mglu = driver.find_element_by_class_name("price-template__text").text
        preco_mglu = transformar_texto(preco_mglu)
    except:
        print('Produto () não disponível no Magazine Luiza.'.format(linha['Link Produto']))
        preco_mglu = linha['Preço Original'] * 3       

    # armazenar menor preço e local na tabela    
    preco_original = linha['Preço Original']
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_mglu, 'Magazine Luiza'), (preco_original, 'Preço Original')]
    lista_precos.sort()
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0] # 0 é o primeiro item da lista, com o menor preço, 0 é o preço
    produtos.loc[i, 'Local'] = lista_precos[0][1] # 0 é o primeiro item da lista, com o menor preço, 1 é o local
    
    # checar se o menor preço está com o desconto desejado comparado ao preço original
    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True


In [47]:
display(produtos)

,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,3887.4,Magazine Luiza
1,"Smart TV LED PRO 43"" Full HD LG 43LM631C0SB",https://www.amazon.com.br/Smart-LG-3LM631C0SB-...,https://www.americanas.com.br/produto/13444474...,https://www.magazineluiza.com.br/smart-tv-led-...,1979,1979,Preço Original


In [50]:
# salvar o arquivo
produtos.to_excel('Produtos.xlsx')

# enviar o e-mail
import yagmail

if enviar_email:
    usuario = yagmail.SMTP(user='seu-email', password='sua-senha') # coloque seu e-mail e sua senha

    to = 'destinatario@gmail.com' # coloque o e-mail do destinatario
    
    subject = f'Produto(s) encontrado(s) com mais de {desconto_min:.0%} de desconto'
    
    # filtrar a tabela de produtos
    
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]
    
    contents = f'''
    <html>
    <head></head>
    <body>
    <p>Bom dia,</p>
    
    <p>Esses são os produtos com mais de {desconto_min:.0%} de desconto:</p>
    
    {tabela_filtrada.to_html()}
    
    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att, Paulo</p>
    </body>
    </html>
    '''
    usuario.send(to, subject, contents)
    
    print('Email enviado!')
    
# finalizando o código
print("Fim da análise")
driver.quit()

Email enviado!
Fim da análise
